<a href="https://colab.research.google.com/github/Zilleplus/MachineLearning/blob/feature%2Fadd-HML-chap15-notebooks/HML/chapter15/examplRNNRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
print("using tensorflow version: " + tf.__version__)


def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2, = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)


n_steps = 50
n_samples = 10000
series = generate_time_series(batch_size=n_samples, n_steps=n_steps + 1)
x_train, y_train = series[:7000, :n_steps], series[:7000, -1]
x_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_steps], series[9000:, -1]

using tensorflow version: 2.7.0


In [ ]:
# Every sample is a serie of 50 values (x) with 1 prediction (y) at the end.
plt.plot(x_train[0])
plt.show()

In [ ]:
def compile_and_fit(model, learning_rate=1e-3, epochs=20):
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse",
                  optimizer=optimizer, metrics=["mse"])
    model.fit(x_train, y_train, epochs=epochs)
    [loss, accuracy] = model.test_on_batch(x=x_test, y=y_test)
    print("loss=" + str(loss) + " accuracy=" + str(accuracy))
    return (loss, accuracy)

In [13]:
# Native forcasting with linear regression model.
model_linear_regression = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])
linear_loss, linear_accuracy = compile_and_fit(model_linear_regression)

Epoch 1/20
219/219 [==============================] - 1s 4ms/step - loss: 0.2006 - mse: 0.2006
Epoch 2/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0610 - mse: 0.0610
Epoch 3/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0296 - mse: 0.0296
Epoch 4/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0194 - mse: 0.0194
Epoch 5/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0153 - mse: 0.0153
Epoch 6/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0131 - mse: 0.0131
Epoch 7/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115
Epoch 8/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102
Epoch 9/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090
Epoch 10/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0081 - mse: 0.0081
Epoch 11/20
219/219 [============================

In [14]:
f_x = np.expand_dims(x_test[0], axis=0)  # batch of 1 dimension
f_y = y_test[0]
f_pred = model_linear_regression.predict(f_x)
print((f_pred - f_y)*(f_pred - f_y))

[[0.01215654]]


In [15]:
model_single_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1]),
])
single_rnn_loss, single_rnn_accuracy = compile_and_fit(model_single_rnn)
# single rnn weights:
# single_rnn_layer = model_single_rnn.layers[0]
# single_rnn_layer.weights[0] => input matrix W_x
# single_rnn_layer.weights[1] => recurrent_matrix W_y
# single_rnn_layer.weights[2] => bias matrix

Epoch 1/20
219/219 [==============================] - 3s 8ms/step - loss: 0.2288 - mse: 0.2288
Epoch 2/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1644 - mse: 0.1644
Epoch 3/20
219/219 [==============================] - 2s 8ms/step - loss: 0.1425 - mse: 0.1425
Epoch 4/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1389 - mse: 0.1389
Epoch 5/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1393 - mse: 0.1393
Epoch 6/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1392 - mse: 0.1392
Epoch 7/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1391 - mse: 0.1391
Epoch 8/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1391 - mse: 0.1391
Epoch 9/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1387 - mse: 0.1387
Epoch 10/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1406 - mse: 0.1406
Epoch 11/20
219/219 [============================

In [17]:
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.Dense(1),
])
rnn_loss, rnn_accuracy = compile_and_fit(model_rnn, learning_rate=1e-3, epochs=20)

Epoch 1/20
219/219 [==============================] - 5s 18ms/step - loss: 0.1471 - mse: 0.1471
Epoch 2/20
219/219 [==============================] - 4s 19ms/step - loss: 0.1234 - mse: 0.1234
Epoch 3/20
219/219 [==============================] - 4s 18ms/step - loss: 0.1118 - mse: 0.1118
Epoch 4/20
219/219 [==============================] - 4s 18ms/step - loss: 0.1070 - mse: 0.1070
Epoch 5/20
219/219 [==============================] - 4s 17ms/step - loss: 0.1051 - mse: 0.1051
Epoch 6/20
219/219 [==============================] - 4s 18ms/step - loss: 0.1035 - mse: 0.1035
Epoch 7/20
219/219 [==============================] - 4s 18ms/step - loss: 0.1016 - mse: 0.1016
Epoch 8/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0990 - mse: 0.0990
Epoch 9/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0975 - mse: 0.0975
Epoch 10/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0948 - mse: 0.0948
Epoch 11/20
219/219 [==================